In [94]:
import random

class Card:
    def __init__(self, name, cost, power):
        self.name = name
        self.cost = cost
        self.power = power

class CardGame:
    def __init__(self):
        self.turn_number = 0
        self.max_energy = 0  # Starting energy
        self.available_energy = 0  # Energy available for playing cards
        self.hand = []
        self.deck = self.generate_deck()
        self.location1 = []
        self.location2 = []
        self.location3 = []
        self.discarded = []
        
    def generate_deck(self):
        deck = [
            Card("Quicksilver", 1, 2),
            Card("Domino", 2, 3),
            Card("Sif", 3, 5),
            Card("Corvus", 3, 5),
            Card("Jubilee", 4, 1),
            Card("Ghostrider", 4, 3),
            Card("Agatha", 6, 14),
            Card("Blackcat", 4, 9),
            Card("Infinaut", 6, 20),
            Card("Red Hulk", 6, 11),
            Card("Odin", 6, 8),
            Card("Hela", 6, 6)
        ]
        random.shuffle(deck)
        domino_card = None
        for card in deck:
            if card.name == "Domino":
                deck.remove(card)
                deck.append(card)
        # If "Domino" is found, remove it from the deck and append it to the end
        if domino_card:
            deck.remove(domino_card)
            deck.append(domino_card)
        return deck

    def draw_card(self, card_name=None, num_cards=1):
            if card_name:
                # Draw a specific card by name
                for card in self.deck:
                    if card.name == card_name:
                        self.hand.append(card)
                        self.deck.remove(card)
                        print(f"Drawn {card_name}")
                        return card
                print(f"Card {card_name} not found in the deck.")
            else:
                # Draw specified number of cards
                if len(self.deck) >= num_cards:
                    drawn_cards = self.deck[:num_cards]
                    self.hand.extend(drawn_cards)
                    del self.deck[:num_cards]
                    print(f"Drawn {num_cards} cards")
                    return drawn_cards
                else:
                    print("Not enough cards in the deck!")

    def play_cards(self):
        if len(self.hand) > 0:
            print("Playing cards...")
            agatha_in_hand = any(card.name == "Agatha" for card in self.hand)
            available_locations = [self.location1, self.location2, self.location3]
            locations_with_space = [loc for loc in available_locations if len(loc) < 4]
            if agatha_in_hand:
                while True:
                    # Create a list of playable cards
                    playable_cards = [card for card in self.hand if card.cost <= self.available_energy]
                    
                    if not playable_cards:
                        break  # No playable cards left, exit the loop
                    
                    # Shuffle the list of playable cards
                    random.shuffle(playable_cards)
                    
                    # Choose a random playable card
                    agatha_playable = any(card.name == "Agatha" for card in playable_cards)
                    if agatha_playable:
                        card_to_play = next((card for card in playable_cards if card.name == "Agatha"), None)
                    else:
                        card_to_play = random.choice(playable_cards)
                    
                    # Play the chosen card
                    
                    self.available_energy -= card_to_play.cost
                    self.hand.remove(card_to_play)
                    chosen_location = random.choice(locations_with_space)
                    self.place_card(card_to_play, location=chosen_location)
            else: 
                # Play cards manually
                #some logic need to decide which
                pass

        else:
            print("Hand is empty!")

    def place_card(self, card, location):
        
        if len(location) < 4:
            location.append(card)
            print(f"Played {card.name} with power {card.power}")
            self.trigger_on_reveal(card, location)
        else:
            print("Location full")

    def trigger_on_reveal(self, card, location):
        if card.name == "Sif":
            max_cost = max(card.cost for card in self.hand)
            highest_cost_cards = [card for card in self.hand if card.cost == max_cost]
            print("Sif effect triggered.")
            self.discard_card(cards=highest_cost_cards)
            
        elif card.name == "Corvus":
            print("Corvus effect triggered.")
            if len(self.hand) >= 2:
                self.add_energy(1)
            self.discard_card()
            self.discard_card()
        elif card.name == "Jubilee":
            if self.deck and len(location) < 4:
                print("Jubilee effect triggered.")
                top_card = self.deck.pop(0)
                self.place_card(top_card, location)
        elif card.name == "Ghostrider":
            if self.discarded and len(location) < 4:
                random_discarded_card = random.choice(self.discarded)
                self.place_card(random_discarded_card, location)
                self.discarded.remove(random_discarded_card)
                #print(f"Ghostrider effect triggered. Ressurected {random_discarded_card.name}.")
        elif card.name == "Odin":
            print("Odin effect triggered.")
            for other_card in location:
                if other_card != card:
                    self.trigger_on_reveal(other_card, location)
        elif card.name == "Hela":
            for discarded_card in self.discarded:
                available_locations = [self.location1, self.location2, self.location3]
                locations_with_space = [loc for loc in available_locations if len(loc) < 4]
                if locations_with_space:
                    random_location = random.choice(locations_with_space)
                    self.discarded.remove(discarded_card)
                    self.place_card(discarded_card, random_location)
                    
                    print(f"Hela effect triggered. Placed {discarded_card.name} to a random location.")
                else:
                    print("Hela effect not triggered. All locations are full.")
        elif card.name == "Blade":
            if self.hand:
                last_card = self.hand[-1]
                self.discard_card(cards=[last_card])
                print("Blade effect triggered. Discarded the last card in hand.")
            else:
                print("Blade effect not triggered. Hand is empty.")
        elif card.name == "Ironlad":
            if self.deck:
                top_card = self.deck[0]
                self.trigger_on_reveal(top_card, location)
                print("Ironlad effect triggered. Triggered on-reveal effect for the top card of the deck.")
            else:
                print("Ironlad effect not triggered. Deck is empty.")
        else:
            print("No on-reveal")


    def play_next_turn(self):
        if self.turn_number < 6:
            self.turn_number += 1
            self.max_energy += 1  # Reset energy to 1 for next turn
            self.available_energy = self.max_energy
            if self.turn_number == 2:
                self.draw_card("Domino")
            else:
                self.draw_card()
            print("Hand:", [card.name for card in self.hand])
            print(f"Start of turn {self.turn_number}, energy: {self.max_energy}")
            self.play_cards()
            for card in self.hand:
                if card.name == 'Blackcat':
                    self.discard_card(cards = [card])
        else:
            print("GAME OVER")


    def discard_card(self, cards=[]):
        if not cards:
            print("discarding randomly")
            cards = self.hand
        if self.hand:
            random_card = random.choice(cards)
            self.hand.remove(random_card)
            self.discarded.append(random_card)
            print(f"Discarded {random_card.name} from hand.")
        else:
            print("Hand is already empty.")

    def play_top_card_from_deck(self):
        if self.deck:
            top_card = self.deck.pop(0)
            print(f"Played top card from deck: {top_card.name}")
        else:
            print("Deck is empty.")

    def add_energy(self, amount):
        self.max_energy += amount
        print(f"Added {amount} energy.")




In [95]:
# Example of using the CardGame class
game = CardGame()

def simulate_game(num_games):
    for game_num in range(1, num_games + 1):
        print(f"\n--- Game {game_num} ---")
        game = CardGame()
        for card in game.deck:
                if card.name == "Agatha":
                    game.draw_card("Agatha")
        game.draw_card("Quicksilver")
        game.draw_card()
        game.draw_card()
        for turn in range(1, 7):  # Simulate 6 turns for each game
            print(f"\n--- Turn {turn} ---")
            
            game.play_next_turn()

# Example usage:
simulate_game(3)  # Simulate 3 games


--- Game 1 ---
Drawn Agatha
Drawn Quicksilver
Drawn 1 cards
Drawn 1 cards

--- Turn 1 ---
Drawn 1 cards
Hand: ['Agatha', 'Quicksilver', 'Red Hulk', 'Infinaut', 'Ghostrider']
Start of turn 1, energy: 1
Playing cards...
Played Quicksilver with power 2
No on-reveal

--- Turn 2 ---
Drawn Domino
Hand: ['Agatha', 'Red Hulk', 'Infinaut', 'Ghostrider', 'Domino']
Start of turn 2, energy: 2
Playing cards...
Played Domino with power 3
No on-reveal

--- Turn 3 ---
Drawn 1 cards
Hand: ['Agatha', 'Red Hulk', 'Infinaut', 'Ghostrider', 'Corvus']
Start of turn 3, energy: 3
Playing cards...
Played Corvus with power 5
Corvus effect triggered.
Added 1 energy.
discarding randomly
Discarded Agatha from hand.
discarding randomly
Discarded Red Hulk from hand.

--- Turn 4 ---
Drawn 1 cards
Hand: ['Infinaut', 'Ghostrider', 'Jubilee']
Start of turn 4, energy: 5
Playing cards...

--- Turn 5 ---
Drawn 1 cards
Hand: ['Infinaut', 'Ghostrider', 'Jubilee', 'Sif']
Start of turn 5, energy: 6
Playing cards...

--- Turn 